In [ ]:
from instruction import keypair_from_json
from processor import mint_nft
from solana.keypair import Keypair
from solana.rpc.async_api import AsyncClient as Client
from state import ClassEnum, rpc_url
from rich import print
client = Client(rpc_url.target_network)
client_state = await client.is_connected()
print("Client is connected" if client_state else "Client is Disconnected")
payer_keypair = keypair_from_json("./keypair.json")
mints = [Keypair() for _ in range(1)]
print([i.public_key for i in mints])
for mint_keypair in mints:
    client = Client(rpc_url.target_network)
    res = await mint_nft(payer_keypair, mint_keypair, ClassEnum.enum.Benitoite(), client)
    print(res)